In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

%matplotlib inline

Using Theano backend.


In [2]:
#Загружаем подготовленные данные

df_parsed = pd.read_csv('data_parsed.csv').drop(['comment'], axis=1).dropna()

#Удаляем объекты с дробными оценками
keep = {'reting': [1.0, 2.0, 3.0, 4.0, 5.0]}
df_parsed = df_parsed[df_parsed[list(keep)].isin(keep).all(axis=1)]

In [3]:
#Разделим выборку на обучающую и тестовую
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(df_parsed['parsed'],
                                                                df_parsed['reting'], test_size=0.2,
                                                                random_state=241)
#Предобработка данных для подачи на вход нейросети
max_features = 20000
maxlen=128
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train_lstm)
X_train_lstm = tokenizer.texts_to_sequences(X_train_lstm)
X_train_lstm = pad_sequences(X_train_lstm, maxlen=maxlen)
X_test_lstm = tokenizer.texts_to_sequences(X_test_lstm)
X_test_lstm = pad_sequences(X_test_lstm, maxlen=maxlen)

y_train_lstm = pd.get_dummies(y_train_lstm).as_matrix()
y_test_lstm = pd.get_dummies(y_test_lstm).as_matrix()

In [4]:
#Нейросеть с LSTM слоем для обработки текстов
model = Sequential()

model.add(Embedding(max_features, 128, dropout=0.3))
model.add(LSTM(128, dropout_U=0.2, dropout_W=0.3))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics = ['accuracy'])

In [5]:
# Обучение модели занимает некоторое время, рекомендуется загрузить уже обученную модель.
# Для загрузки модели раскомментируйте следующую строку.
model = load_model('lstm.h5')

# Для обучения модели раскомментируете строки ниже.
# batch_size = 32
# model.fit(X_train_lstm, y_train_lstm, nb_epoch = 7, batch_size=batch_size, verbose = 1,
#          validation_split=0.2)

In [ ]:
# Сохранение модели
# model.save('lstm.h5')
# del model

In [7]:
batch_size = 32
# train_loss, train_acc = model.evaluate(X_train_lstm, y_train_lstm, batch_size=batch_size)
test_loss, test_acc = model.evaluate(X_test_lstm, y_test_lstm, batch_size=batch_size)

# print("\n\naccuracy на обучающей выборке:\t", train_acc)
print("\n\naccuracy на текстовой выборке:\t:", test_acc)

3072/3077 [============================>.] - ETA: 0s

accuracy на текстовой выборке:	: 0.588235294127
